In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import *
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk
from nltk import word_tokenize, pos_tag, RegexpParser
from nltk.corpus import stopwords
import sys

###import into pandas dataframe

In [2]:
rawdf = pd.read_csv('training_set.tsv', sep = '\t')

In [7]:
# rawvaldf= pd.read_csv('validation_set.tsv', sep = '\t')

###define phrases to parse

####reuse and edit code used in othe project

In [166]:
#noun phrase
npatterns = """
    NP: {<NN.*><IN>*<NN.*>+}
    {<JJ.*>*<NN.*>+}
    """

#{<JJ.*>*<NN.*><CC>*<NN.*>+}
#verb phrase patterns
vpatterns = """
    VP: {<VB.*><VB.*>*}
    """
# {<VB.*>+<DT|IN>*<JJ.*>*<NN.*>+<R.*>*}
#adjective phrase patterns
apatterns = """
    AP: {<JJ.*><IN><NN.*>}
    {<RB.*><JJ.*>}
    {<JJ.*>+}
    """

In [167]:
NPChunker = RegexpParser(npatterns)
VPChunker = RegexpParser(vpatterns)
APChunker = RegexpParser(apatterns)

In [32]:
def prepare_text(input):
    """Takes a list of sentences and returns words that have their parts of speech tagged"""
    tokenized_words = [word_tokenize(sent) for sent in input]
    tagged_words = [pos_tag(word) for word in tokenized_words]
    return tagged_words


def make_tree(tagged_words, typetree = 'nps'):
    """takes POS tagged words and returns word tree"""
    if typetree =='nps':
        word_tree = [NPChunker.parse(word) for word in tagged_words]
    if typetree =='vps':
        word_tree = [VPChunker.parse(word) for word in tagged_words]
    if typetree == 'aps':
        word_tree = [APChunker.parse(word) for word in tagged_words]
    return word_tree


def return_a_list_of_Ps(input, ptype ='nps'):
    """takes list of sentences, calls prepare_text, make_tree; parses it returns set of unique phrases 
    depending on the phrase type
    """
    ps = []  # an empty list in which to phrases will be stored.
    t_words = prepare_text(input)
    p_tree = make_tree(t_words, ptype)
#     print p_tree
    for sent in p_tree:
        if ptype =='nps':
            tree = NPChunker.parse(sent)
        if ptype =='vps':
            tree = VPChunker.parse(sent)
        if ptype =='aps':
            tree = APChunker.parse(sent)

        for subtree in tree.subtrees():
            if subtree.label() == ptype.upper()[:2]:
                t = subtree
                t = ' '.join(word for word, tag in t.leaves())
                ps.append(t)
    return list(set(ps))

In [41]:
# rawdf['question_sent'] = rawdf['question'].map(lambda x: [x.encode('utf-8','ignore') for x in x])
rawdf['question_sent'] = rawdf['question'].map(lambda x: sent_tokenize(x.decode("utf8")))

In [168]:
rawdf['qkeyps'] = rawdf['question_sent'].map(lambda x: list(set(return_a_list_of_Ps(x)+return_a_list_of_Ps(x,'vps')+return_a_list_of_Ps(x,'aps'))))

In [178]:
def flatten(inputlist):
    t_copy = word_tokenize(u' '.join(inputlist))
    dups = list(set([x for x in t_copy if t_copy.count(x) > 1]))
    for i in dups:
        while i in inputlist:
            inputlist.remove(i)
    return inputlist

In [179]:
rawdf['u_qkeyps'] = rawdf['qkeyps'].map(lambda x: flatten(x))

In [ ]:
i = 105
print rawdf.u_qkeyps[i]
print rawdf.question[i]
rawdf[i:i+1]